# Making an "Environment" class<a class="tocSkip"></a>

Thomas Schanzer z5310829  
School of Physics, UNSW  
September 2021

In this notebook, we create an $\verb |Environment|$ class to group all the functions associated with a particular atmospheric sounding. This will make it easier to handle several different soundings at once.

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#The-new-class" data-toc-modified-id="The-new-class-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>The new class</a></span></li><li><span><a href="#Testing" data-toc-modified-id="Testing-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Testing</a></span></li></ul></div>

In [1]:
import numpy as np
import pandas as pd

import metpy.calc as mpcalc
import metpy.constants as const
from metpy.units import units
from metpy.units import concatenate

from scipy.interpolate import interp1d
from scipy.integrate import solve_ivp

import sys

## The new class

In [59]:
class Environment():
    def __init__(self, pressure, temperature, dewpoint):
        self.pressure_raw = pressure.to(units.mbar)
        self.temperature_raw = temperature.to(units.celsius)
        self.dewpoint_raw = dewpoint.to(units.celsius)
        
        self.temperature_interp = interp1d(
            self.pressure_raw.m, self.temperature_raw.m, kind='linear')
        self.dewpoint_interp = interp1d(
            self.pressure_raw.m, self.dewpoint_raw.m, kind='linear')
        
    def temperature_from_pressure(self, pressure):
        """
        Finds the environmental temperature at a given pressure.
        """

        temperature = self.temperature_interp(pressure.to(units.mbar).m)
        return temperature*units.celsius

    def dewpoint_from_pressure(self, pressure):
        """
        Finds the environmental dew point at a given pressure.
        """

        dewpoint = self.dewpoint_interp(pressure.to(units.mbar).m)
        return dewpoint*units.celsius

## Testing

In [60]:
file = '../soundings/SYDNEY AIRPORT (94767) 12 Nov 2019 00Z.txt'
sounding = pd.read_csv(
    file, names=['pressure', 'temperature', 'dewpoint'],
    usecols=[0, 2, 3], header=0)
sounding = sounding.to_numpy()
pressure = sounding[:,0]*units.mbar
temperature = sounding[:,1]*units.celsius
dewpoint = sounding[:,2]*units.celsius

sydney = Environment(pressure, temperature, dewpoint)

In [64]:
sydney.temperature_from_pressure(500.8*units.mbar)

-13.02 <Unit('degree_Celsius')>

In [65]:
sydney.dewpoint_from_pressure(500.8*units.mbar)

-23.02 <Unit('degree_Celsius')>